In [42]:
import torch
# Load fine-tuned model
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny")
model = AutoModelForSequenceClassification.from_pretrained("prajjwal1/bert-tiny")

trained = torch.load('./notebooks/SST-2-BERT-tiny.bin', map_location=torch.device("cpu"))
trained.pop('bert.embeddings.position_ids', None) # Remove unexpected keys
model.load_state_dict(trained , strict=True)

model.eval()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/var/folders/3t/qv8zrd4d5t12dqxjq4rxf56r0000gn/T/ipykernel_68915/3930920598.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user vi

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-1

In [32]:
65536/4

16384.0

In [41]:
print(layer0_intermediate_weight[-2])
print(layer0_intermediate_weight4[-2])

-0.03560466319322586
-0.03560466319322586


In [31]:
if 0.09879531 in layer0_intermediate_weight:
    print("yes")
else:
    print("no")

no


In [68]:
model.bert.encoder.layer[0].intermediate.dense.weight.transpose(0, 1)[31][511]

tensor(-0.2465, grad_fn=<SelectBackward0>)

In [67]:
model.bert.encoder.layer[0].intermediate.dense.weight.transpose(0, 1)[127][511]

tensor(0.1565, grad_fn=<SelectBackward0>)

In [2]:
layer0_intermediate_weight = model.bert.encoder.layer[0].intermediate.dense.weight.transpose(0, 1).double().detach().numpy()
layer1_intermediate_weight = model.bert.encoder.layer[1].intermediate.dense.weight.transpose(0, 1).double().detach().numpy()

In [3]:
layer0_output_weight = model.bert.encoder.layer[0].output.dense.weight.transpose(0, 1).double().detach().numpy()
layer1_output_weight = model.bert.encoder.layer[1].output.dense.weight.transpose(0, 1).double().detach().numpy()

In [4]:
len(layer0_intermediate_weight[0])*len(layer0_intermediate_weight)

65536

In [5]:
print(layer0_intermediate_weight[:5])

[[ 9.87953097e-02  3.80718447e-02  2.70095393e-02 ...  1.55190434e-02
  -4.06939769e-03  2.84967363e-01]
 [ 2.60726884e-02  5.67517877e-02 -2.42611510e-04 ... -2.85039637e-02
   3.91533189e-02 -1.56141698e-01]
 [ 6.57312619e-03  9.66107659e-03  7.95733277e-03 ... -3.60841379e-02
   3.38025503e-02  1.83659047e-01]
 [-4.64717075e-02 -1.24046348e-01 -4.05563377e-02 ...  5.58397286e-02
  -8.19457620e-02  6.64943233e-02]
 [-2.24055313e-02 -7.50428289e-02  1.97503995e-02 ... -1.78632364e-01
   6.18782192e-02  4.11385410e-02]]


In [44]:
import os
import numpy as np

# Create a directory to save the parameters
os.makedirs('new-weights-sst2', exist_ok=True)

# Access the state dictionary of the model
state_dict = model.state_dict()

# Function to save a tensor to a text file
def save_tensor_to_txt_flat(tensor, filepath):
    print("flat")
    # Convert tensor to numpy array
    array = tensor.cpu().detach().numpy()
    
    # Flatten the array in column-major (Fortran-style) order
    flattened = array.T.flatten()
    
    # Calculate how many full rows of 128 elements we can make
    num_full_rows = len(flattened) // 128
    
    # Reshape the array to have 128 columns, dropping any extra elements
    reshaped_array = flattened[:num_full_rows * 128].reshape(-1, 128)
    
    # Save to text file with 128 values per line
    np.savetxt(filepath, reshaped_array, delimiter=',')
    
    # If there are any remaining elements, save them on a final line
    remaining = flattened[num_full_rows * 128:]
    if len(remaining) > 0:
        with open(filepath, 'a') as f:
            np.savetxt(f, remaining.reshape(1, -1), delimiter=',')
# Selfoutput
def save_tensor_to_txt_flat_F(tensor, filepath):
    print("flat-F")
    # Convert tensor to numpy array
    array = tensor.cpu().detach().numpy()
    
    # Flatten the array in column-major (Fortran-style) order
    flattened = array.T.flatten('F')
    
    # Calculate how many full rows of 128 elements we can make
    num_full_rows = len(flattened) // 128
    
    # Reshape the array to have 128 columns, dropping any extra elements
    reshaped_array = flattened[:num_full_rows * 128].reshape(-1, 128)
    
    # Save to text file with 128 values per line
    np.savetxt(filepath, reshaped_array, delimiter=',')
    
    # If there are any remaining elements, save them on a final line
    remaining = flattened[num_full_rows * 128:]
    if len(remaining) > 0:
        with open(filepath, 'a') as f:
            np.savetxt(f, remaining.reshape(1, -1), delimiter=',')
def save_tensor_to_txt(tensor, filepath):
    print("origin")
    # Convert tensor to numpy array
    array = tensor.cpu().numpy()
    # Save to text file with desired formatting
    with open(filepath, 'w') as f:
        for row in array.reshape(-1):
            f.write(f"{row}\n")
    #np.savetxt(filepath, array, delimiter=',')
            
# Function to save a tensor to a text file
def save_tensor_to_txt2(tensor, filepath):
    print("intermediate/output")
    # Convert tensor to numpy array
    array = tensor.cpu().numpy()
    # Reshape the array to 128 rows and 128 columns
    reshaped_array = array.reshape(128, 128)
    # Save to text file with desired formatting
    np.savetxt(filepath, reshaped_array, delimiter=',')

exclude_keywords = [
    "attself_query_weight",
    "attself_key_weight",
    "attself_value_weight",
    "selfoutput_weight",
    "pooler_dense_weight",
]
    
# Iterate through all parameters and save the ones you're interested in
for name, param in state_dict.items():
    # Check if the parameter name matches the desired layers
    if any(keyword in name for keyword in [
        'attention.self.query', 
        'attention.self.key', 
        'attention.self.value',
        'attention.output.dense',
        'intermediate.dense',
        'output.dense',
        'pooler.dense',
        'classifier'
    ]):
        # Replace dots with underscores for file naming
        filename = name.replace('.', '_') + '.txt'
        
        # remove bert_encoder_
        if filename[:13] == "bert_encoder_":
            filename = filename[13:]
        elif filename[:5] == "bert_":
            filename = filename[5:]
            
        # layer_0 -> layer0
        filename = filename.replace('layer_0', 'layer0')
        filename = filename.replace('layer_1', 'layer1')
        
        # attention_self -> attself
        # attention_output_dense -> selfoutput
        # intermediate_dense -> intermediate
        # output_dense -> output
        filename = filename.replace('attention_self', 'attself')
        filename = filename.replace('attention_output_dense', 'selfoutput')
        filename = filename.replace('intermediate_dense', 'intermediate')
        filename = filename.replace('output_dense', 'output')
        
        filepath = os.path.join('new-weights-sst2', filename)
        
        if any(keyword in filepath for keyword in exclude_keywords):
            if "selfoutput" in filepath:
                save_tensor_to_txt_flat_F(param, filepath)
            elif "weight" in filepath:
                save_tensor_to_txt_flat(param, filepath)
            else:
                save_tensor_to_txt2(param, filepath)
        elif "intermediate_weight" in filepath or "output_weight" in filepath:
            save_tensor_to_txt_flat(param, filepath)
        else:
            save_tensor_to_txt(param, filepath)
        print(f"Saved {name} to {filepath}")

flat
Saved bert.encoder.layer.0.attention.self.query.weight to new-weights-sst2/layer0_attself_query_weight.txt
origin
Saved bert.encoder.layer.0.attention.self.query.bias to new-weights-sst2/layer0_attself_query_bias.txt
flat
Saved bert.encoder.layer.0.attention.self.key.weight to new-weights-sst2/layer0_attself_key_weight.txt
origin
Saved bert.encoder.layer.0.attention.self.key.bias to new-weights-sst2/layer0_attself_key_bias.txt
flat
Saved bert.encoder.layer.0.attention.self.value.weight to new-weights-sst2/layer0_attself_value_weight.txt
origin
Saved bert.encoder.layer.0.attention.self.value.bias to new-weights-sst2/layer0_attself_value_bias.txt
flat-F
Saved bert.encoder.layer.0.attention.output.dense.weight to new-weights-sst2/layer0_selfoutput_weight.txt
origin
Saved bert.encoder.layer.0.attention.output.dense.bias to new-weights-sst2/layer0_selfoutput_bias.txt
flat
Saved bert.encoder.layer.0.intermediate.dense.weight to new-weights-sst2/layer0_intermediate_weight.txt
origin
Save

In [45]:
len(layer0_intermediate_weight)

65536

In [46]:
len(layer0_intermediate_weight[0])

TypeError: object of type 'numpy.float64' has no len()

In [47]:
import numpy as np

layer0_intermediate_weight = np.loadtxt(f"./new-weights-sst2/layer0_intermediate_weight.txt", delimiter=",")
layer1_intermediate_weight = np.loadtxt(f"./new-weights-sst2/layer1_intermediate_weight.txt", delimiter=",")

layer0_output_weight = np.loadtxt(f"./new-weights-sst2/layer0_output_weight.txt", delimiter=",")
layer1_output_weight = np.loadtxt(f"./new-weights-sst2/layer1_output_weight.txt", delimiter=",")

# Reshape the matrices into single columns
layer0_intermediate_weight = layer0_intermediate_weight.flatten(order='C')
layer1_intermediate_weight = layer1_intermediate_weight.flatten(order='C')
layer0_output_weight = layer0_output_weight.flatten(order='C')
layer1_output_weight = layer1_output_weight.flatten(order='C')

In [49]:
split_size = len(layer0_intermediate_weight)//4
num = int(split_size)

layer0_intermediate_weight1 = layer0_intermediate_weight[:num]
layer0_intermediate_weight2 = layer0_intermediate_weight[num:(num*2)]
layer0_intermediate_weight3 = layer0_intermediate_weight[(num*2):(num*3)]
layer0_intermediate_weight4 = layer0_intermediate_weight[(num*3):(num*4)]
layer1_intermediate_weight1 = layer1_intermediate_weight[:num]
layer1_intermediate_weight2 = layer1_intermediate_weight[num:(num*2)]
layer1_intermediate_weight3 = layer1_intermediate_weight[(num*2):(num*3)]
layer1_intermediate_weight4 = layer1_intermediate_weight[(num*3):(num*4)]

layer0_output_weight1 = layer0_output_weight[:num]
layer0_output_weight2 = layer0_output_weight[num:(num*2)]
layer0_output_weight3 = layer0_output_weight[(num*2):(num*3)]
layer0_output_weight4 = layer0_output_weight[(num*3):(num*4)]
layer1_output_weight1 = layer1_output_weight[:num]
layer1_output_weight2 = layer1_output_weight[num:(num*2)]
layer1_output_weight3 = layer1_output_weight[(num*2):(num*3)]
layer1_output_weight4 = layer1_output_weight[(num*3):(num*4)]

np.savetxt("./new-weights-sst2/layer0_intermediate_weight1.txt", layer0_intermediate_weight1, delimiter=",")
np.savetxt("./new-weights-sst2/layer0_intermediate_weight2.txt", layer0_intermediate_weight2, delimiter=",")
np.savetxt("./new-weights-sst2/layer0_intermediate_weight3.txt", layer0_intermediate_weight3, delimiter=",")
np.savetxt("./new-weights-sst2/layer0_intermediate_weight4.txt", layer0_intermediate_weight4, delimiter=",")
np.savetxt("./new-weights-sst2/layer1_intermediate_weight1.txt", layer1_intermediate_weight1, delimiter=",")
np.savetxt("./new-weights-sst2/layer1_intermediate_weight2.txt", layer1_intermediate_weight2, delimiter=",")
np.savetxt("./new-weights-sst2/layer1_intermediate_weight3.txt", layer1_intermediate_weight3, delimiter=",")
np.savetxt("./new-weights-sst2/layer1_intermediate_weight4.txt", layer1_intermediate_weight4, delimiter=",")

np.savetxt("./new-weights-sst2/layer0_output_weight1.txt", layer0_output_weight1, delimiter=",")
np.savetxt("./new-weights-sst2/layer0_output_weight2.txt", layer0_output_weight2, delimiter=",")
np.savetxt("./new-weights-sst2/layer0_output_weight3.txt", layer0_output_weight3, delimiter=",")
np.savetxt("./new-weights-sst2/layer0_output_weight4.txt", layer0_output_weight4, delimiter=",")
np.savetxt("./new-weights-sst2/layer1_output_weight1.txt", layer1_output_weight1, delimiter=",")
np.savetxt("./new-weights-sst2/layer1_output_weight2.txt", layer1_output_weight2, delimiter=",")
np.savetxt("./new-weights-sst2/layer1_output_weight3.txt", layer1_output_weight3, delimiter=",")
np.savetxt("./new-weights-sst2/layer1_output_weight4.txt", layer1_output_weight4, delimiter=",")

In [15]:
test = np.loadtxt("./new-weights-sst2/layer0_intermediate_weight1.txt", delimiter=",")

In [16]:
len(test)

16384

In [17]:
os.remove("./new-weights-sst2/layer0_intermediate_weight.txt")
os.remove("./new-weights-sst2/layer1_intermediate_weight.txt")

os.remove("./new-weights-sst2/layer0_output_weight.txt")
os.remove("./new-weights-sst2/layer1_output_weight.txt")

In [18]:
# Verify weights
saved_weight = np.loadtxt('./new-weights-sst2/layer0_intermediate_weight1.txt')
model_weight = model.bert.encoder.layer[0].intermediate.dense.weight.detach().numpy().flatten()[:len(saved_weight)]

print("Saved weight (first 5):", saved_weight[:5])
print("Model weight (first 5):", model_weight[:5])
print("Are they equal?", np.allclose(saved_weight, model_weight))

Saved weight (first 5): [0.09879531 0.03807184 0.02700954 0.03793886 0.01012474]
Model weight (first 5): [ 0.09879531  0.02607269  0.00657313 -0.04647171 -0.02240553]
Are they equal? False


In [19]:
print(len(layer0_intermediate_weight1))
print(len(layer0_intermediate_weight2))
print(len(layer0_intermediate_weight3))
print(len(layer0_intermediate_weight4))

16384
16384
16384
16384


In [20]:
print(layer0_intermediate_weight1[16383])
print(layer0_intermediate_weight2[0])
print(layer0_intermediate_weight2[16383])
print(layer0_intermediate_weight3[16383])
print(layer0_intermediate_weight4[16383])

-0.2465175986289978
-0.00038853593287058175
-0.22945359349250793
-0.11301503330469131
0.15645954012870789


In [21]:
print(layer0_intermediate_weight[16383])
print(layer0_intermediate_weight[16384])
print(layer0_intermediate_weight[32767])
print(layer0_intermediate_weight[49151])
print(layer0_intermediate_weight[65535])

-0.2465175986289978
-0.00038853593287058175
-0.22945359349250793
-0.11301503330469131
0.15645954012870789
